In [1]:
using System;
using System.Linq;
using System.Text.Json;
using System.Collections.Generic;
using System.Text.Json.Serialization;
using System.Text.Json.Serialization.Metadata;

In [2]:
class ExampleClass
{
    public string Name { get; set; } = "";
    public int Age {get;set;} = 10;
    public SecretHolder? Secret { get; set; }
    public bool HasError { get; set; } = false;
}

class SecretHolder
{
    public string Value { get; set; } = "";
}

In [5]:
class IgnorePropertiesWithName
{
    private readonly string[] _ignoredNames;

    public IgnorePropertiesWithName(params string[] ignoredNames)
        => _ignoredNames = ignoredNames;

    public void ModifyTypeInfo(JsonTypeInfo ti)
    {
        if (ti.Kind != JsonTypeInfoKind.Object)
            return;
            
        foreach(var property in ti.Properties.ToList())
        {
            if(_ignoredNames.Contains(property.Name))
            {
                ti.Properties.Remove(property);
            }
        }
    }
}

public class IgnoreTypeExample
{
    public static void RunIt()
    {
        var modifier = new IgnorePropertiesWithName("HasError");

        JsonSerializerOptions options = new()
        {
            TypeInfoResolver = new DefaultJsonTypeInfoResolver
            {
                Modifiers = { modifier.ModifyTypeInfo }
            }
        };

        ExampleClass obj = new()
        {
            Name = "Password",
            Secret = new SecretHolder { Value = "MySecret" },
            HasError = true
        };

        string output = JsonSerializer.Serialize(obj, options);
        Console.WriteLine(output);
        // {"Name":"Password","Secret":{"Value":"MySecret"}}
    }
}



In [11]:
IgnoreTypeExample.RunIt();

{"Name":"Password","Age":10,"Secret":{"Value":"MySecret"}}
